In [1]:
import pandas as pd
import numpy as np
import scipy as sp


## actionType对应的actionTimeDiff

+ 统计1-9分别对应的时间差，最近5个时间差作为提取特征。

+ 这里进行分类计算，[1],[2,3,4],[5,6,7,8,9]

In [2]:
df_train_data = pd.read_csv('../data/train/orderFuture_train.csv')
df_test_data = pd.read_csv('../data/test/orderFuture_test.csv')

# 首先选择action信息
df_train_action = pd.read_csv('../data/train/action_train.csv')
df_test_action = pd.read_csv('../data/test/action_test.csv')


In [3]:
def diffInfo(x):
    if len(x) == 0:
        return x
    
    # 所有时间的最大值
    cur = 1505087865
    
    x = list(x)
    x.append(cur)
#     print(x)
    for i in range(len(x)-1):
        x[i] = x[i+1] - x[i]
    
    return np.array(x[:-1])


In [4]:
# 时间间隔特征，[avg,std,var,skew,kurt,min,max, last1, last2,last3,last4,last5]
def getTypeKLast5Diff(df_train_action, Types):
    '''参数：
        df_train_action: 应用数据，可以是train和test
        Type: actionType的值
    
    '''
    Type = ""
    for i in Types:
        Type += str(i)
    
    print(Type)
    column = ['userid','type'+Type+'DiffTimeAvg','type'+Type+'DiffTimeMin','type'+Type+'DiffTimeMax',
              'type'+Type+'Last1DiffTime','type'+Type+'Last2DiffTime','type'+Type+'Last3DiffTime',
              'type'+Type+'Last4DiffTime','type'+Type+'Last5DiffTime',]

    df_train_diffTime = pd.DataFrame(columns=column)

    userlist = list(set(df_train_action['userid']))

    k = 0
    for user in userlist:
        k+=1
        if k%5000 == 0:
            print(k)

        df_users_t = df_train_action[df_train_action['userid'] == user]

        # 选择type k 对应的数据
        df_users_t = df_users_t[df_users_t['actionType'].isin(Types)]

        df_users_t = df_users_t.sort_values('actionTime')


        df_users_t['actionTimeDiff'] = diffInfo(df_users_t['actionTime'])

        row = [user]

        ## 所有的差值的最值和平均值
        row.append(df_users_t['actionTimeDiff'].mean())
        row.append(df_users_t['actionTimeDiff'].min())
        row.append(df_users_t['actionTimeDiff'].max())

        # 注意要时间进行发转，统计的时候是到这统计的。
        tmp_time = []
        tmp_time.extend(df_users_t['actionTimeDiff'].values[-5:])
        tmp_time.reverse()

        # 用差值的平均数填充
        if len(tmp_time) < 5:
            tmp_time.extend([row[1] for i in range(5-len(tmp_time))])

        row.extend(tmp_time)

        df_train_diffTime = df_train_diffTime.append(pd.DataFrame([row],columns=column),ignore_index=True)

    # 极差(max-min),(max-avg),(avg-min)
    df_train_diffTime['typeDiff'+Type+'TimeMaxSubMin'] = df_train_diffTime['type'+Type+'DiffTimeMax'] - df_train_diffTime['type'+Type+'DiffTimeMin']
    df_train_diffTime['typeDiff'+Type+'TimeMaxSubAvg'] = df_train_diffTime['type'+Type+'DiffTimeMax'] - df_train_diffTime['type'+Type+'DiffTimeAvg']
    df_train_diffTime['typeDiff'+Type+'TimeAvgSubMin'] = df_train_diffTime['type'+Type+'DiffTimeAvg'] - df_train_diffTime['type'+Type+'DiffTimeMin']
        
    return df_train_diffTime



In [5]:
## train
for Types in [[1],[2,3,4],[5,6,7,8,9]]:
    print(Types)
    df_train_diffTime = getTypeKLast5Diff(df_train_action,Types)
    df_train_data = pd.merge(df_train_data,df_train_diffTime, how='left',on='userid')


[1]
1
5000
10000
15000
20000
25000
30000
35000
40000
[2, 3, 4]
234
5000
10000
15000
20000
25000
30000
35000
40000
[5, 6, 7, 8, 9]
56789
5000
10000
15000
20000
25000
30000
35000
40000


In [6]:
## test
for Types in [[1],[2,3,4],[5,6,7,8,9]]:
    print(Types)
    df_test_diffTime = getTypeKLast5Diff(df_test_action,Types)
    df_test_data = pd.merge(df_test_data,df_test_diffTime, how='left',on='userid')


[1]
1
5000
10000
[2, 3, 4]
234
5000
10000
[5, 6, 7, 8, 9]
56789
5000
10000


In [10]:
df_train_data.info()
df_test_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 40307 entries, 0 to 40306
Data columns (total 34 columns):
userid                        40307 non-null int64
type1DiffTimeAvg              40307 non-null float64
type1DiffTimeMin              40307 non-null int64
type1DiffTimeMax              40307 non-null int64
type1Last1DiffTime            40307 non-null int64
type1Last2DiffTime            40307 non-null float64
type1Last3DiffTime            40307 non-null float64
type1Last4DiffTime            40307 non-null float64
type1Last5DiffTime            40307 non-null float64
typeDiff1TimeMaxSubMin        40307 non-null int64
typeDiff1TimeMaxSubAvg        40307 non-null float64
typeDiff1TimeAvgSubMin        40307 non-null float64
type234DiffTimeAvg            40307 non-null float64
type234DiffTimeMin            40307 non-null int64
type234DiffTimeMax            40307 non-null int64
type234Last1DiffTime          40307 non-null int64
type234Last2DiffTime          40307 non-null float64
type23

In [11]:
df_train_data.fillna(-1,inplace=True)
df_test_data.fillna(-1,inplace=True)


In [14]:
# df_train_data.drop('orderType',axis=1,inplace=True)
df_train_data.to_csv('../data/dataSet/df_train_3.csv',index=False,index_label=False)
df_test_data.to_csv('../data/dataSet/df_test_3.csv',index=False,index_label=False)
